In [15]:
from requests_html import HTMLSession

session = HTMLSession()
portatil_links = []

for i in range(1,21):
    print(f'Pagina {i}')
    r = session.get(url=f"https://www.pcbox.com/ordenadores/ordenadores-portatiles?page={i}")
    
    # Get all links with the specified class
    links = r.html.find('a.vtex-product-summary-2-x-clearLink.vtex-product-summary-2-x-clearLink--main-product-summary.h-100.flex.flex-column')
    # Extract absolute URLs and remove duplicates
    links = list(set([link.absolute_links.pop() for link in links]))
    portatil_links.extend(links)

Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20


In [16]:
portatil_links

['https://www.pcbox.com/82fg01r1sp-ip-s500-i5-1135g7-8gb-512gb/p',
 'https://www.pcbox.com/a30ytea-abe-omen-17-r7-8845hs-32gb-ddr5-2dm-5600-1tb-rtx-4070-8gb-vram-avid-qhd/p',
 'https://www.pcbox.com/b4xl6ea-abe-16-r1047ns-roaree-24c1-victus-16-freedos-core-i7-14650hx-32gb-ddr5-/p',
 'https://www.pcbox.com/9v7h4ea-t-cs-15-fa0047ns-i7-12650h-16gb-512gb-fdos/p',
 'https://www.pcbox.com/a30yzea-abe-omen-17-r7-8845hs-32gb-ddr5-2dm-5600-1tb-rtx-4060-8gb-vram-avid-ant/p',
 'https://www.pcbox.com/a30ymea-abe-omen-16-i9-14900hx-32gb-ddr5-2dm-5600-1tb-gen4-rtx-4060-8gb-vram-br/p',
 'https://www.pcbox.com/ga403ui-qs049-portatil-asus-ga403ui-qs049/p',
 'https://www.pcbox.com/984g1ea-abe-15-fa1012nsi7-13700h16-140608dos/p',
 'https://www.pcbox.com/83a58ea-portatil-hp-victus-gaming-16-r0009ns-i7-13700h-16gb-512gb-16--rtx-4050-/p',
 'https://www.pcbox.com/am1c8ea-portatil-hp-victus-15-fb2010ns-r7-8845hs-16gb-1tb-rtx-4050-15-6--freedo/p',
 'https://www.pcbox.com/a31bmea-portatil-hp-victus-16-r1026ns-i

In [20]:
import sys
import json
import requests
from bs4 import BeautifulSoup

def scrape_laptop_specs(url):
    try:
        # Realiza la petición HTTP
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad responses (4xx, 5xx)

    except requests.RequestException as e:
        print(f"Error al realizar la petición HTTP: {e}")
        return None

    try:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Característiques del portatil
        sections = {}

        # URL del producto
        sections["URL"] = url
        
        # Titul del portatil
        title = soup.find("h1", class_="vtex-store-components-3-x-productNameContainer vtex-store-components-3-x-productNameContainer--quickview mv0 t-heading-4")
        if title:
            sections["Titulo"] = title.text
        
        # Marca
        marca = soup.find("span", class_="ticnova-commons-components-0-x-ProductInfo c-action-primary ml2")
        if marca:
            sections['Marca'] = marca.text

        # Se buscan todas las tablas con la clase utilizada en las especificaciones
        tables = soup.find_all("table", class_="vtex-table-description-extended_carac")
        
        # Iteramos sobre cada tabla
        for table in tables:
            current_section = None
            # Iteramos sobre las filas que contienen la clase indicativa
            for row in table.find_all("tr", class_="vtex-table-description-row"):
                try:
                    # Si la fila tiene un <td> con colspan="2" y un div de título, es un encabezado de sección
                    header_td = row.find("td", colspan="2")
                    if header_td:
                        title_div = header_td.find("div", class_="vtex-table-description-title")
                        if title_div:
                            current_section = title_div.get_text(strip=True)
                            sections.setdefault(current_section, {})
                            continue

                    # Si la fila contiene clave y valor, los extraemos
                    key_td = row.find("td", class_="vtex-table-description-key")
                    value_td = row.find("td", class_="vtex-table-description-value")
                    if key_td and value_td and current_section:
                        key = key_td.get_text(strip=True)
                        value = value_td.get_text(strip=True)
                        sections[current_section][key] = value
                except Exception as e:
                    print(f"Error al procesar una fila de la tabla: {e}")
                    continue

        # Mapeamos las secciones extraídas a los campos requeridos:
        mapping = {
            "Procesador": "Procesador",
            "Memoria": "RAM",
            "Medios de almacenaje": "Almacenamiento",
            "Gráficos": "Graficos",
            "Exhibición": "Pantalla",
            "Software": "Sistema Operativo",
            "Batería": "Bateria"
        }
        
        try:
            price = soup.find('div', class_='ticnova-commons-components-0-x-price')
            if price:
                sections['Precio'] = price.text.replace('\xa0', '').replace('€', '').strip()
        except Exception as e:
            print(f"Error al extraer el precio: {e}")

        result = {}
        for seccion_original, campo in mapping.items():
            result[campo] = sections.get(seccion_original, {})

        return sections

    except Exception as e:
        print(f"Error general al procesar la página: {e}")
        return None

In [26]:
all_laptops_specs = []

for link in portatil_links:
    if any(json.loads(specs_str).get("URL") == link for specs_str in all_laptops_specs):
        print(f"Skipping {link} - already processed")
        continue
    try:
        specs = scrape_laptop_specs(link)
        if specs is not None:
            try:
                specs_json = json.dumps(specs, indent=4, ensure_ascii=False)
                all_laptops_specs.append(specs_json)
            except json.JSONEncodeError as e:
                print(f"Error encoding JSON for {link}: {e}")
    except Exception as e:
        print(f"Error processing link {link}: {e}")
        continue

KeyboardInterrupt: 

In [24]:
for laptop in all_laptops_specs:
    print(laptop)

{
    "URL": "https://www.pcbox.com/82fg01r1sp-ip-s500-i5-1135g7-8gb-512gb/p",
    "Titulo": "PORTATIL LENOVO IP 5 15ITL05 CORE i5-1165G7 2.4GHZ /8GB/512GB SSD/15.6\" FHD /FREEDOS ",
    "Marca": "LENOVO",
    "Procesador": {
        "Frecuencia del procesador": "2,4 GHz",
        "Fabricante de procesador": "Intel",
        "Frecuencia del procesador turbo": "4,2 GHz",
        "Modelo del procesador": "i5-1135G7",
        "Número de núcleos de procesador": "4",
        "Chipset": "Intel® SoC",
        "Familia de procesador": "Intel® Core™ i5",
        "Potencia de diseño térmico configurable-baja": "12 W",
        "Frecuencia de potencia de diseño térmico configurable-baja": "0,9 GHz",
        "Frecuencia de potencia de diseño térmico configurable-alta": "2,4 GHz",
        "Potencia de diseño térmico configurable-alta": "28 W",
        "Generación del procesador": "Intel® Core™ i5 de 11ma Generación"
    },
    "Memoria": {
        "Forma de factor de memoria": "Incorporado",
       

In [25]:
from pymongo import MongoClient
import json

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['pcbox']
collection = db['portatiles']

# Parse each specs_json into a dict and insert into MongoDB
for laptop_specs_json in all_laptops_specs:
    # Parse JSON string into dict if needed
    if isinstance(laptop_specs_json, str):
        laptop_specs_dict = json.loads(laptop_specs_json)
    else:
        laptop_specs_dict = laptop_specs_json
    
    # Insert document
    collection.insert_one(laptop_specs_dict)

In [40]:
import csv
import re
import json

def simplify_procesador(proc_dict):
    fabricante = proc_dict.get("Fabricante de procesador", "").strip()
    familia = proc_dict.get("Familia de procesador", "").strip()
    modelo = proc_dict.get("Modelo del procesador", "").strip()
    # Eliminar símbolos de marca (® y ™)
    familia_clean = re.sub(r"[®™]", "", familia).strip()
    # Si el fabricante no está incluido, lo anteponemos
    if fabricante and fabricante not in familia_clean:
        familia_clean = f"{fabricante} {familia_clean}".strip()
    # Si el modelo empieza con el mismo token final de la familia, lo eliminamos para evitar duplicados
    last_token = familia_clean.split()[-1]
    if modelo.startswith(last_token):
        new_model = modelo[len(last_token):]
        # Si new_model comienza con guión, lo concatenamos directamente
        if new_model.startswith("-"):
            final = f"{familia_clean}{new_model}"
        else:
            final = f"{familia_clean}-{modelo}"
    else:
        final = f"{familia_clean}-{modelo}"
    return final

def simplify_ram(ram_dict):
    # Extrae el número de GB desde "Memoria interna" (por ejemplo, "8 GB")
    ram_value = ram_dict.get("Memoria interna", "")
    match = re.search(r"(\d+)", ram_value)
    return match.group(1) if match else ""

def simplify_tipo_ram(ram_dict):
    tipo_ram = ram_dict.get("Tipo de memoria interna", "")
    # Extrae la parte DDR (por ejemplo, "DDR4" de "DDR4-SDRAM")
    match = re.search(r"(DDR\d+)", tipo_ram, re.IGNORECASE)
    return match.group(1).upper() if match else ""

def simplify_almacenamiento(alm_dict):
    # Se prefiere "Capacidad total de SSD", pero si no existe se usa "SDD, capacidad"
    valor = alm_dict.get("Capacidad total de SSD", "") or alm_dict.get("SDD, capacidad", "")
    match = re.search(r"(\d+)", valor)
    return match.group(1) if match else ""

def simplify_graficos(graf_dict):
    return graf_dict.get("Modelo de adaptador gráfico incorporado", "").strip()

def simplify_pantalla(pant_dict):
    diag = pant_dict.get("Diagonal de la pantalla", "")
    # Se extrae la medida en pulgadas que aparece entre paréntesis, por ejemplo: (15.6")
    match = re.search(r"\(([\d\.]+)\"", diag)
    return match.group(1) if match else ""

def simplify_resolucion(pant_dict):
    resol = pant_dict.get("Resolución de la pantalla", "")
    # Se buscan los dos números (ancho y alto)
    numbers = re.findall(r"(\d+)", resol)
    if len(numbers) >= 2:
        return f"{numbers[0]}x{numbers[1]}"
    return ""

def simplify_sistema(soft_dict):
    return soft_dict.get("Sistema operativo instalado", "").strip()

def simplify_bateria(bat_dict):
    bat = bat_dict.get("Capacidad de batería", "")
    match = re.search(r"(\d+)", bat)
    return match.group(1) if match else ""

def simplify_specs(specs):
    simplified = {}
    simplified["Procesador"] = simplify_procesador(specs.get("Procesador", {}))
    simplified["RAM"] = simplify_ram(specs.get("Memoria", {}))
    simplified["Tipo RAM"] = simplify_tipo_ram(specs.get("Memoria", {}))
    simplified["Almacenamiento"] = simplify_almacenamiento(specs.get("Medios de almacenaje", {}))
    simplified["Graficos"] = simplify_graficos(specs.get("Gráficos", {}))
    simplified["Pantalla"] = simplify_pantalla(specs.get("Exhibición", {}))
    simplified["Resolucion"] = simplify_resolucion(specs.get("Exhibición", {}))
    simplified["Sistema Operativo"] = simplify_sistema(specs.get("Software", {}))
    simplified["Bateria"] = simplify_bateria(specs.get("Batería", {}))
    simplified["Precio"] = specs.get("Precio", "")
    return simplified

In [32]:
if all_laptops_specs:
    first_laptop_specs = json.loads(all_laptops_specs[0])
    precio = first_laptop_specs.get("Precio", "No precio disponible")
    print(f"Precio del primer laptop: {precio}")
    
    # Print other key details from the first laptop
    print("\nOther details from the first laptop:")
    if 'Marca' in first_laptop_specs:
        print(f"Marca: {first_laptop_specs['Marca']}")
    if 'Procesador' in first_laptop_specs:
        procesador = first_laptop_specs['Procesador']
        print(f"Modelo del procesador: {procesador.get('Modelo del procesador', 'No disponible')}")
    if 'Memoria' in first_laptop_specs:
        memoria = first_laptop_specs['Memoria']
        print(f"Memoria RAM: {memoria.get('Memoria interna', 'No disponible')}")
    if 'Medios de almacenaje' in first_laptop_specs:
        almacenamiento = first_laptop_specs['Medios de almacenaje']
        print(f"Almacenamiento: {almacenamiento.get('Capacidad total de SSD', 'No disponible')}")

else:
    print("No laptop specifications available.")

Precio del primer laptop: 549,00

Other details from the first laptop:
Marca: LENOVO
Modelo del procesador: i5-1135G7
Memoria RAM: 8 GB
Almacenamiento: 512 GB


In [29]:
dataset_portatils = []

# Process each laptop in all_laptops_specs
for laptop_specs_json in all_laptops_specs:
    # Convert JSON string to dictionary
    laptop_specs_dict = json.loads(laptop_specs_json)
    # Simplify specs and add to dataset
    simplified = simplify_specs(laptop_specs_dict)
    dataset_portatils.append(simplified)
    
# Definición de las columnas del CSV en el orden deseado
fieldnames = [
    "Procesador",
    "RAM",
    "Tipo RAM",
    "Almacenamiento",
    "Graficos",
    "Pantalla",
    "Resolucion",
    "Sistema Operativo",
    "Bateria",
    "Precio"
]

In [30]:
# Write all simplified specs to CSV
with open("specs_simplified.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for laptop in dataset_portatils:
        writer.writerow(laptop)

In [42]:
import csv
import json
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['pcbox']
collection = db['portatiles']

# Retrieve all laptops from MongoDB
mongo_laptops = list(collection.find())
print(f"Retrieved {len(mongo_laptops)} laptops from MongoDB")

# Track URLs to avoid duplicates
processed_urls = set()
dataset_portatils = []

# Process each laptop from MongoDB
for laptop_specs_dict in mongo_laptops:
    # Skip if this URL has already been processed
    url = laptop_specs_dict.get('URL', '')
    if url in processed_urls:
        print(f"Skipping duplicate URL: {url}")
        continue
    
    # Add URL to processed set
    processed_urls.add(url)
    
    try:
        # Simplify specs and add to dataset
        simplified = simplify_specs(laptop_specs_dict)
        dataset_portatils.append(simplified)
    except Exception as e:
        print(f"Error processing laptop {url}: {e}")

print(f"Total unique laptops after deduplication: {len(dataset_portatils)}")

# Check for duplicates in the existing CSV file if it exists
existing_urls = set()
try:
    with open("specs_simplified.csv", "r", newline="", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # We don't have URLs in our simplified data, so create a signature based on other fields
            laptop_signature = f"{row['Procesador']}_{row['RAM']}_{row['Almacenamiento']}_{row['Precio']}"
            existing_urls.add(laptop_signature)
except FileNotFoundError:
    print("No existing CSV file found, will create a new one")

# Remove laptops that already exist in the CSV
filtered_dataset = []
for laptop in dataset_portatils:
    laptop_signature = f"{laptop['Procesador']}_{laptop['RAM']}_{laptop['Almacenamiento']}_{laptop['Precio']}"
    if laptop_signature not in existing_urls:
        filtered_dataset.append(laptop)
    else:
        print(f"Skipping laptop already in CSV: {laptop_signature}")

print(f"Adding {len(filtered_dataset)} new laptops to CSV")

# Write all simplified specs to CSV (append mode to add to existing file if it exists)
with open("specs_simplified.csv", "a", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write header only if the file is new (empty)
    if csvfile.tell() == 0:
        writer.writeheader()
    
    # Write rows
    for laptop in filtered_dataset:
        writer.writerow(laptop)

print("CSV update complete!")

Retrieved 260 laptops from MongoDB
Error processing laptop https://www.pcbox.com/6a3m5av--%3E%3Eidsuma-i5-1135g7-ti-pd-ic-830g8b/p: list index out of range
Error processing laptop https://www.pcbox.com/456f6ec-abe-pb-640-g8-i5-1145g7-16-256-lte-w10p/p: list index out of range
Error processing laptop https://www.pcbox.com/a3yu3et-abe-zbpowerg11-u7-155h-32gb-1tb-rtx-w11p/p: list index out of range
Error processing laptop https://www.pcbox.com/98k26et-abe-zbfury-i9-14900hx-64gb-1tb-rtx-w11p/p: list index out of range
Error processing laptop https://www.pcbox.com/zlg-00037-laptop6-5ultra-16g-256g-15--plat-w11p/p: list index out of range
Error processing laptop https://www.pcbox.com/zlp-00037-microsoft-laptop6-i7-16gb-256gb-15-sp-plat/p: list index out of range
Error processing laptop https://www.pcbox.com/zjw-00012-microsoft-laptop6-i7-16gb-512gb-13-sp-bk/p: list index out of range
Error processing laptop https://www.pcbox.com/zlq-00037-microsoft-laptop6-i7-16gb-512gb-15-sp-plat/p: list in